In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import floor

In [33]:
def f(x):
    d = [1.22, 1.18, 2.5, 2.35, 4.47, 6.02, 7.35, 10.9, 15.33]
    i = floor((2*x-2))
    return d[i]

In [34]:
def create_matrix_and_res(x, y):
    A = np.empty((m,m))
    b = np.empty(m)
    
    for i in range(m):
        for j in range(m):
            A[i][j] = sum([x[k]**(i + j) for k in range(n + 1)])
        b[i] = sum([y[k] * x[k] ** i for k in range(n + 1)])
        
    return A, b

def find_alphas(A, b):
    return (np.linalg.inv(A).dot(b.T)).T

def find_alphas2(A, b):
    T = np.zeros((m,m))
    x = np.empty(m)
    y = np.empty(m)
    
    # вычисляем t 
    for i in range(m):
        for j in range(i):
            T[i][j] = (A[i][j] - sum([T[i][k] * T[j][k] for k in range(j)])) / T[j][j]
        T[i][i] = np.sqrt(A[i][i] - sum(T[i][k] ** 2 for k in range(i)))
        
    # обратный ход
    for i in range(m):
        y[i] = (b[i] - sum([T[i,k]*y[k] for k in range(i)])) / T[i,i] 
    
    for i in range(m-1, -1, -1):
        x[i] = (y[i] - sum([T[k,i]*x[k] for k in range(i+1, m)])) / T[i,i] 

    return x
    

In [62]:
m = 4

x0 = 1
xn = 5
n = 8
h = (xn-x0)/n
xs = np.linspace(x0, xn, n+1, True)
ys = [f(x) for x in xs]

A, b = create_matrix_and_res(xs, ys)

alphas = find_alphas2(A,b)
print("alphas:")
print(alphas)

MSE = np.sqrt(
        sum([
            (ys[k] - sum([alphas[i] * xs[k]**i for i in range(m)]))
             for k in range(m+1)
         ])**2
    ) / np.sqrt(n)
print("MSE")
print(MSE)

delta = MSE / np.sqrt(sum([ys[k]**2 for k in range(n+1)]))
print("delta")
print(delta)


def f_my(x):
    return sum([alphas[i] * x**i for i in range(m)])

xs_test = np.linspace(x0, xn, n+n+1, True)
res_my = np.vectorize(f_my)(xs_test)
res_true = list(np.vectorize(f)(xs_test))

# plt.plot(xs_test, res_my)
# plt.scatter(xs, res_true)

df = pd.DataFrame({"x": xs_test, "f(x)": res_true, "approx": res_my,  "abs_diff": np.abs(res_true-res_my)})
for i, _ in enumerate(res_true):
    if i%2 == 1:
        df.iloc[i, 1] = pd.NA
        df.iloc[i, 3] = pd.NA
        
df

alphas:
[-0.74071429  2.5434127  -0.99744589  0.22505051]
MSE
0.019162287663323653
delta
0.0008758069434217044


,x,f(x),approx,abs_diff
0,1.00,1.22,1.030303,0.189697
1,1.25,NaN,1.319594,NaN
2,1.50,1.18,1.589697,0.409697
3,1.75,NaN,1.861710,NaN
4,2.00,2.50,2.156732,0.343268
5,2.25,NaN,2.495860,NaN
6,2.50,2.35,2.900195,0.550195
7,2.75,NaN,3.390833,NaN
8,3.00,4.47,3.988874,0.481126
9,3.25,NaN,4.715417,NaN
